Notebook to sketch out implementation of QAOA algorithm in pytorch.

Final impementation in `k_sat/pytorch_solver/`

In [1]:
import torch
import numpy as np

In [2]:
from math import log
def t_to_sv(t):
  t = t.detach().clone()
  # For easier visualisation
  t.real[torch.abs(t.real) < 1e-6] = 0
  t.imag[torch.abs(t.imag) < 1e-6] = 0
  return Statevector(t.numpy())


In [3]:
from qiskit.quantum_info.states.statevector import Statevector 
real = torch.tensor([1., 0.])
imag = torch.tensor([0., 0.])
a = torch.complex(real, imag)
t_to_sv(a).draw("latex")

<IPython.core.display.Latex object>

Application of cost unitary:

In [4]:
def cost(z, h, g_j):
  hg = torch.complex(torch.tensor(0.), h * g_j)
  hg_exp = torch.exp(hg)
  z = hg_exp * z
  return z

Application of mixing unitary:

In [5]:
def mix(z, g_j):
  cg = torch.complex(torch.cos(g_j), torch.tensor(0.))
  sg = torch.complex(torch.tensor(0.), torch.sin(g_j))

  for i in range(n):
    cz = cg * z

    # swap indices
    z = z.reshape((2,) * n)
    z = z.transpose(0, i)
    fh, sh = z.split(1)
    z = torch.cat((sh, fh))
    z = z.transpose(0, i)
    z = z.reshape(N)

    # reduce for iteration to continue
    z = cz + sg * z
  
  return z



E.g. we'd expect $\ket{000} + \ket{001}$ to be mixed to $\ket{111} + \ket{100}$ when using an angle $\beta_j = \frac{\pi}{2}$

In [6]:
n = 3
N = 2 ** n
real = torch.zeros(size=(N,), dtype=torch.float32)
imag = torch.zeros(size=(N,), dtype=torch.float32)

# |000> + |001> 
real[0] = 1
real[1] = 1
y = torch.complex(real, imag)

ry = mix(y, torch.tensor(torch.pi/2))

In [7]:
t_to_sv(y).draw("latex")

<IPython.core.display.Latex object>

In [8]:
t_to_sv(ry).draw("latex")

<IPython.core.display.Latex object>

Putting this all together:

In [9]:
def qaoa(z, gamma, beta, p, h):

  # Apply layers
  for j in range(p):
    z = cost(z, h, gamma[j])
    z = mix(z, beta[j])
  
  return z

def succ_prob(z, h):
  z = torch.abs(z)
  z = z * z
  return torch.dot(z, h)

In [14]:
p = 3
n = 4
N = 2 ** n

# Starting state
coeff = 1 / (2 ** (n / 2))
s = torch.full((N, ), coeff, dtype=torch.cfloat)

# initial values
gamma_i = torch.full(size=(p, ), fill_value=-0.01)
beta_i = torch.full(size=(p, ), fill_value=0.01)

# optimisable parameters
gamma = torch.nn.Parameter(gamma_i)
beta = torch.nn.Parameter(beta_i)

# h(x) = 1 iff x satisfies formula 
h = torch.zeros(size=(N,))
h[-1] = 1

# Optimisation
epochs = 1000
optimiser = torch.optim.Adam([gamma, beta], lr=0.01, maximize=True)

for i in range(epochs):
  optimiser.zero_grad()
  z = qaoa(s, gamma, beta, p, h)
  p_succ = succ_prob(z, h)
  p_succ.backward()
  optimiser.step()
  if i % 100 == 0:
    print(f'Success probability: {p_succ.item()}')

Success probability: 0.062201038002967834
Success probability: 0.43176916241645813
Success probability: 0.7115023136138916
Success probability: 0.7281084656715393
Success probability: 0.7281786203384399
Success probability: 0.7281787395477295
Success probability: 0.728178858757019
Success probability: 0.7281787395477295
Success probability: 0.7281789183616638
Success probability: 0.7281786203384399


Check output state of circuit

In [15]:
gamma_f = gamma.detach().clone()
beta_f = beta.detach().clone()

output = qaoa(s, gamma_f, beta_f, p, h)

In [16]:
t_to_sv(output).draw("latex")

<IPython.core.display.Latex object>

In [19]:
t_to_sv(output).probabilities_dict(decimals=3)

{'0000': 0.01,
 '0001': 0.025,
 '0010': 0.025,
 '0011': 0.024,
 '0100': 0.025,
 '0101': 0.024,
 '0110': 0.024,
 '0111': 0.005,
 '1000': 0.025,
 '1001': 0.024,
 '1010': 0.024,
 '1011': 0.005,
 '1100': 0.024,
 '1101': 0.005,
 '1110': 0.005,
 '1111': 0.728}